In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
df_postulantes_edu_original = pd.read_csv('Originales/fiuba_1_postulantes_educacion.csv')
df_postulantes_datos_original = pd.read_csv('Originales/fiuba_2_postulantes_genero_y_edad.csv')
df_postulantes_vistas_original = pd.read_csv('Originales/fiuba_3_vistas.csv')
df_postulaciones_original = pd.read_csv('Originales/fiuba_4_postulaciones.csv')
df_avisos_online_original = pd.read_csv('Originales/fiuba_5_avisos_online.csv')
df_avisos_detalles_original = pd.read_csv('Originales/fiuba_6_avisos_detalle.csv')

In [3]:
df_postulantes_edu_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_1_postulantes_educacion.csv')
df_postulantes_datos_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_2_postulantes_genero_y_edad.csv')
df_postulantes_vistas_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_3_vistas.csv')
df_postulaciones_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_4_postulaciones.csv')
df_avisos_online_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_5_avisos_online.csv')
df_avisos_detalles_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_6_avisos_detalle.csv')

In [4]:
df_postulantes_edu_desde = pd.read_csv('Desde 15 abril/fiuba_1_postulantes_educacion.csv')
df_postulantes_datos_desde = pd.read_csv('Desde 15 abril/fiuba_2_postulantes_genero_y_edad.csv')
df_postulantes_vistas_desde = pd.read_csv('Desde 15 abril/fiuba_3_vistas.csv')
df_avisos_detalles_desde = pd.read_csv('Desde 15 abril/fiuba_6_avisos_detalle.csv')

In [4]:
df_avisos_detalles_faltantes = pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

# Detalles de avisos

Veamos el tamaño de los detalles de avisos para generar un dataset para entrenar

In [5]:
df_avisos_detalles_original.shape

(13534, 11)

In [6]:
df_avisos_detalles_nuevo.shape

(18299, 11)

In [8]:
df_avisos_detalles_faltantes.shape

(338, 11)

Juntamos todo

In [5]:
df_avisos_detalles = pd.concat([df_avisos_detalles_original, df_avisos_detalles_nuevo, df_avisos_detalles_faltantes])

In [6]:
df_avisos_detalles.shape

(32171, 11)

Eliminamos duplicados (No tiene sentido que hayan avisos exactamente iguales, es decir, mismo Id de aviso)

In [7]:
df_avisos_detalles = df_avisos_detalles.drop_duplicates(subset='idaviso')

In [11]:
df_avisos_detalles.shape

(19696, 11)

Veamos las columnas utiles

In [12]:
df_avisos_detalles.columns

Index(['idaviso', 'idpais', 'titulo', 'descripcion', 'nombre_zona', 'ciudad',
       'mapacalle', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',
       'denominacion_empresa'],
      dtype='object')

Descartamos las que tengan mayoria de valors nulos, no aportan información relevante

In [13]:
df_avisos_detalles['mapacalle'].isnull().sum()

18052

In [14]:
df_avisos_detalles['ciudad'].isnull().sum()

19598

In [8]:
del df_avisos_detalles['mapacalle']
del df_avisos_detalles['ciudad']

El pais es único y está en todos los datos, no aporta nada

In [16]:
df_avisos_detalles['idpais'].nunique()

1

In [17]:
df_avisos_detalles['idpais'].isnull().sum()

0

In [9]:
del df_avisos_detalles['idpais']

In [19]:
df_avisos_detalles['nombre_zona'].isnull().sum()

0

In [20]:
df_avisos_detalles['nombre_zona'].value_counts()

Gran Buenos Aires              18022
Capital Federal                 1645
Buenos Aires (fuera de GBA)       17
GBA Oeste                         10
Corrientes                         1
Cordoba                            1
Name: nombre_zona, dtype: int64

In [10]:
del df_avisos_detalles['nombre_zona']

Nos quedamos con

In [22]:
df_avisos_detalles.columns

Index(['idaviso', 'titulo', 'descripcion', 'tipo_de_trabajo', 'nivel_laboral',
       'nombre_area', 'denominacion_empresa'],
      dtype='object')

Parseamos el titulo y la descripción para sacar datos relevantes y generar nuevos features

In [11]:
def lowercase_descripcion(row):
    if pd.isnull(row['descripcion']):
        return row['descripcion']   
    
    return row['descripcion'].lower()

def lowercase_titulo(row):
    if pd.isnull(row['titulo']):
        return row['titulo']   
    
    return row['titulo'].lower()

In [12]:
df_avisos_detalles['descripcion'] = df_avisos_detalles.apply(lowercase_descripcion, axis=1)
df_avisos_detalles['titulo'] = df_avisos_detalles.apply(lowercase_titulo, axis=1)

In [13]:
df_avisos_detalles.head()

,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,vendedor/a provincia de santa fe,<p><strong><strong>empresa:</strong></strong> ...,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,enfermeras,<p>solicitamos para importante cadena de farma...,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,chofer de taxi,<p>te gusta manejar? queres ganar plata hacien...,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,chofer de camioneta bahia blanca - punta alta,<p><strong>somos una empresa multinacional que...,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,operarios de planta - rubro electrodomésticos,<p><strong>operarios de planta</strong></p><p>...,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [14]:
def extra(row, list_of_words):
    if pd.isnull(row['descripcion']) and pd.isnull(row['titulo']):
        return 0
    for word in list_of_words:
        if word in row['descripcion'] or word in row['titulo']:
            return 1
    return 0

In [15]:
pal_claves = {
    'web' : ['web', 'pagina', 'página'],
    'crecimiento' : ['crecim', 'progre'],
    'capacitacion' : ['aprender', 'aprendiza', 'capacitar', 'capacitacion', 'capacitación'],
    'clima laboral' : ['excelente cl', 'clima'],
    'multinacional' : ['multin', 'internacional'],
    'incorp_inmediata' : ['incorporación', 'incorporacion'],
    'disponibilidad_horaria' : ['fines de sem', 'sábado', 'sabado', 'domingo', 'feriado', 'rotativo'],
    'disponibilidad_viajar' : ['viaje', 'viajar'],
    'flexibilidad' : ['flexi'],
    'secundario' : ['secundario compl'],
    'terciario' : ['terciario'],
    'ingles' : ['inglés', 'ingles'],
    'estudiantes' : ['universi', 'estudiant'],
    'conoc_informatica' : ['excel', 'word', 'office'],
    'experiencia' : ['años de exp', 'experiencia'],
    'proactivo' : ['proactiv'],
    'relaciones_pers' : ['relaciones', 'personales'],
    'desarrollador' : ['desarr'],
    'cajero' : ['cajero'],
    'vendedor' : ['vended'],
    'ingeniero' : ['ingeni'],
    'analista' : ['analis'],
    'administrativo' : ['administra'],
    'junior' : ['jr', 'junior'],
    'jefe' : ['jefe'],
    'marketing' : ['marke', 'tele'],
    'supervisor' : ['superv']
}

for k, v in pal_claves.items():
    df_avisos_detalles[k] = df_avisos_detalles.apply((lambda x: extra(x, v)), axis=1)

In [16]:
del df_avisos_detalles['descripcion']
del df_avisos_detalles['titulo']

In [17]:
df_avisos_detalles.shape

(19696, 32)

In [18]:
df_avisos_detalles.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,...,desarrollador,cajero,vendedor,ingeniero,analista,administrativo,junior,jefe,marketing,supervisor
0,8725750,Full-time,Senior / Semi-Senior,Comercial,VENTOR,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,17903700,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000150677,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000610287,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1000872556,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
list_tipos_trabajos = []
list_nivel_laboral = []
list_nombre_area = []
list_empresa = []


def tipos_trabajos(row):
    if pd.isnull(row['tipo_de_trabajo']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['tipo_de_trabajo'] in list_tipos_trabajos:
        return list_tipos_trabajos.index(row['tipo_de_trabajo'])+1
    
    list_tipos_trabajos.append(row['tipo_de_trabajo'])
    
    return list_tipos_trabajos.index(row['tipo_de_trabajo'])+1

def nivel_laboral(row):
    if pd.isnull(row['nivel_laboral']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['nivel_laboral'] in list_nivel_laboral:
        return list_nivel_laboral.index(row['nivel_laboral'])+1
    
    list_nivel_laboral.append(row['nivel_laboral'])
    
    return list_nivel_laboral.index(row['nivel_laboral'])+1

def nombre_area(row):
    if pd.isnull(row['nombre_area']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['nombre_area'] in list_nombre_area:
        return list_nombre_area.index(row['nombre_area'])+1
    
    list_nombre_area.append(row['nombre_area'])
    
    return list_nombre_area.index(row['nombre_area'])+1

def nombre_empresa(row):
    if pd.isnull(row['denominacion_empresa']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['denominacion_empresa'] in list_empresa:
        return list_empresa.index(row['denominacion_empresa'])+1
    
    list_empresa.append(row['denominacion_empresa'])
    
    return list_empresa.index(row['denominacion_empresa'])+1

df_avisos_detalles['tipo_de_trabajo'] = df_avisos_detalles.apply(tipos_trabajos, axis=1)
df_avisos_detalles['nivel_laboral'] = df_avisos_detalles.apply(nivel_laboral, axis=1)
df_avisos_detalles['nombre_area'] = df_avisos_detalles.apply(nombre_area, axis=1)
df_avisos_detalles['denominacion_empresa'] = df_avisos_detalles.apply(nombre_empresa, axis=1)

In [20]:
df_avisos_detalles.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,...,desarrollador,cajero,vendedor,ingeniero,analista,administrativo,junior,jefe,marketing,supervisor
0,8725750,1,1,1,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,17903700,1,1,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000150677,1,1,3,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000610287,1,1,3,4,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1000872556,1,1,4,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
df_avisos_detalles.shape

(19696, 32)

In [22]:
df_avisos_online = pd.concat([df_avisos_online_original, df_avisos_online_nuevo])
df_avisos_online = df_avisos_online.drop_duplicates()

In [23]:
df_avisos_online['online'] = 1
df_avisos_detalles = pd.merge(df_avisos_detalles, df_avisos_online, how='left', on=['idaviso'])

In [24]:
df_avisos_detalles['online'] = df_avisos_detalles['online'].fillna(value=0)

In [25]:
df_avisos_detalles.shape

(19696, 33)

Usar postulaciones para rellenar vistas que no tenga


In [26]:
postulaciones = pd.concat([df_postulaciones_original, df_postulaciones_nuevo])

In [27]:
postulaciones.shape

(8311264, 3)

In [28]:
postulaciones = postulaciones.drop_duplicates(['idaviso', 'idpostulante'])

In [143]:
postulaciones.shape

(6603752, 3)

In [30]:
postulaciones_total = postulaciones.loc[:, ['idaviso', 'idpostulante']].rename(columns={'idpostulante':'postu_total'})\
                .groupby('idaviso').count().reset_index()

In [31]:
df_avisos_detalles = pd.merge(df_avisos_detalles, postulaciones_total, on=['idaviso'], how='left')

In [32]:
df_avisos_detalles.shape

(19696, 34)

In [33]:
vistas = pd.concat([df_postulantes_vistas_original, df_postulantes_vistas_nuevo])

In [34]:
vistas.shape

(6720583, 3)

In [35]:
vistas = vistas.drop_duplicates()

In [36]:
vistas = vistas.rename(columns={'idAviso' : 'idaviso'})
vistas.shape

(6682624, 3)

In [37]:
vistas2 = vistas.loc[:, ['idaviso', 'idpostulante']].groupby('idaviso').count().reset_index()

In [38]:
vistas2 = vistas2.rename(columns={'idpostulante' : 'total_vistas'})

In [39]:
df_avisos_detalles = pd.merge(df_avisos_detalles, vistas2, how='left', on=['idaviso'])

In [40]:
df_avisos_detalles.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,...,ingeniero,analista,administrativo,junior,jefe,marketing,supervisor,online,postu_total,total_vistas
0,8725750,1,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0.0,137.0,NaN
1,17903700,1,1,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,175.0,1935.0
2,1000150677,1,1,3,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,NaN,2.0
3,1000610287,1,1,3,4,0,1,1,0,1,...,0,0,0,0,0,0,0,1.0,197.0,72.0
4,1000872556,1,1,4,5,0,0,0,0,0,...,0,0,0,0,0,0,1,0.0,NaN,1.0


In [41]:
vistas2 = vistas.groupby(['idaviso', 'idpostulante']).count().reset_index()

In [42]:
vistas2 = vistas2.loc[:, ['idaviso', 'timestamp']].groupby('idaviso').median().reset_index()

In [43]:
vistas2 = vistas2.rename(columns={'timestamp' : 'vistas_median'})

In [44]:
df_avisos_detalles = pd.merge(df_avisos_detalles, vistas2, how='left', on=['idaviso'])

In [45]:
df_avisos_detalles.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,...,analista,administrativo,junior,jefe,marketing,supervisor,online,postu_total,total_vistas,vistas_median
0,8725750,1,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0.0,137.0,NaN,NaN
1,17903700,1,1,2,2,0,0,0,0,0,...,0,0,0,0,0,0,1.0,175.0,1935.0,2.0
2,1000150677,1,1,3,3,0,0,0,0,0,...,0,0,0,0,0,0,0.0,NaN,2.0,2.0
3,1000610287,1,1,3,4,0,1,1,0,1,...,0,0,0,0,0,0,1.0,197.0,72.0,2.0
4,1000872556,1,1,4,5,0,0,0,0,0,...,0,0,0,0,0,1,0.0,NaN,1.0,1.0


In [46]:
vistas["timestamp"] = pd.to_datetime(vistas["timestamp"])

In [47]:
df_postulaciones = pd.concat([df_postulaciones_nuevo, df_postulaciones_original])

In [48]:
df_postulaciones.shape

(8311264, 3)

In [49]:
df_postulaciones = df_postulaciones.drop_duplicates(['idaviso', 'idpostulante'])

In [50]:
df_postulaciones.shape

(6603752, 3)

In [51]:
df_postulaciones['fechapostulacion'] = pd.to_datetime(df_postulaciones["fechapostulacion"], errors = 'coerce')

In [52]:
fechas_postulaciones = df_postulaciones.loc[:, ['idaviso', 'fechapostulacion']].groupby('idaviso').min().reset_index()

In [53]:
vistas = vistas.rename(columns={'timestamp' : 'fechapostulacion'})

In [54]:
fechas_vistas_postul = vistas.loc[:, ['idaviso', 'fechapostulacion']].groupby('idaviso').min().reset_index()

In [55]:
fechas_iniciales = pd.concat([fechas_postulaciones, fechas_vistas_postul])

In [56]:
fechas_iniciales.shape

(35266, 2)

In [57]:
fechas_iniciales = fechas_iniciales.groupby('idaviso').min().reset_index()

In [58]:
fechas_iniciales.shape

(21158, 2)

In [59]:
fechas_iniciales = fechas_iniciales.rename(columns={'fechapostulacion' : 'fecha_inicio'})

In [60]:
fechas_postulaciones = df_postulaciones.loc[:, ['idaviso', 'fechapostulacion']].groupby('idaviso').max().reset_index()

In [61]:
fechas_vistas_postul = vistas.loc[:, ['idaviso', 'fechapostulacion']].groupby('idaviso').max().reset_index()

In [62]:
fechas_finales = pd.concat([fechas_postulaciones, fechas_vistas_postul])

In [63]:
fechas_finales.shape

(35266, 2)

In [64]:
fechas_finales = fechas_finales.groupby('idaviso').max().reset_index()

In [65]:
fechas_finales.shape

(21158, 2)

In [66]:
fechas_finales = fechas_finales.rename(columns={'fechapostulacion':'fecha_fin'})

In [67]:
fechas_totales = pd.merge(fechas_finales, fechas_iniciales, on=['idaviso'], how='inner')

In [68]:
fechas_totales.shape

(21158, 3)

In [69]:
fechas_totales.head()

,idaviso,fecha_fin,fecha_inicio
0,1585950,2018-02-26 00:27:07.739,2018-02-26 00:27:07.739
1,1672250,2018-02-27 00:19:37.075,2018-02-26 16:42:16.493
2,1799300,2018-02-25 22:13:34.379,2018-02-25 22:13:34.379
3,2613010,2018-02-28 17:31:59.425,2018-02-26 21:39:47.129
4,3006520,2018-02-27 23:26:44.163,2018-02-27 23:26:44.163


In [70]:
def set_dias(row):
    return row.days

In [71]:
fechas_totales['cant_dias'] = (fechas_totales['fecha_fin']-fechas_totales['fecha_inicio']).apply(set_dias)

In [72]:
fechas_totales.head()

,idaviso,fecha_fin,fecha_inicio,cant_dias
0,1585950,2018-02-26 00:27:07.739,2018-02-26 00:27:07.739,0
1,1672250,2018-02-27 00:19:37.075,2018-02-26 16:42:16.493,0
2,1799300,2018-02-25 22:13:34.379,2018-02-25 22:13:34.379,0
3,2613010,2018-02-28 17:31:59.425,2018-02-26 21:39:47.129,1
4,3006520,2018-02-27 23:26:44.163,2018-02-27 23:26:44.163,0


In [73]:
fechas_totales.head()

,idaviso,fecha_fin,fecha_inicio,cant_dias
0,1585950,2018-02-26 00:27:07.739,2018-02-26 00:27:07.739,0
1,1672250,2018-02-27 00:19:37.075,2018-02-26 16:42:16.493,0
2,1799300,2018-02-25 22:13:34.379,2018-02-25 22:13:34.379,0
3,2613010,2018-02-28 17:31:59.425,2018-02-26 21:39:47.129,1
4,3006520,2018-02-27 23:26:44.163,2018-02-27 23:26:44.163,0


In [74]:
del fechas_totales['fecha_fin']
del fechas_totales['fecha_inicio']

In [75]:
df_avisos_detalles.shape

(19696, 36)

In [76]:
df_avisos_detalles_final = pd.merge(df_avisos_detalles, fechas_totales, on=['idaviso'], how='left')

In [73]:
df_avisos_detalles_final.shape

(19696, 37)

In [74]:
df_avisos_detalles_final.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,...,administrativo,junior,jefe,marketing,supervisor,online,postu_total,total_vistas,vistas_median,cant_dias
0,8725750,1,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0.0,137.0,NaN,NaN,26.0
1,17903700,1,1,2,2,0,0,0,0,0,...,0,0,0,0,0,1.0,175.0,1935.0,2.0,51.0
2,1000150677,1,1,3,3,0,0,0,0,0,...,0,0,0,0,0,0.0,NaN,2.0,2.0,3.0
3,1000610287,1,1,3,4,0,1,1,0,1,...,0,0,0,0,0,1.0,197.0,72.0,2.0,83.0
4,1000872556,1,1,4,5,0,0,0,0,0,...,0,0,0,0,1,0.0,NaN,1.0,1.0,0.0


In [75]:
df_avisos_detalles_final.isnull().sum()

idaviso                      0
tipo_de_trabajo              0
nivel_laboral                0
nombre_area                  0
denominacion_empresa         0
web                          0
crecimiento                  0
capacitacion                 0
clima laboral                0
multinacional                0
incorp_inmediata             0
disponibilidad_horaria       0
disponibilidad_viajar        0
flexibilidad                 0
secundario                   0
terciario                    0
ingles                       0
estudiantes                  0
conoc_informatica            0
experiencia                  0
proactivo                    0
relaciones_pers              0
desarrollador                0
cajero                       0
vendedor                     0
ingeniero                    0
analista                     0
administrativo               0
junior                       0
jefe                         0
marketing                    0
supervisor                   0
online  

In [77]:
a = df_avisos_detalles_final['vistas_median'].median()
a

1.0

In [78]:
def fill_total_vistas(row):
    if((np.isnan(row['total_vistas'])) & (np.isnan(row['vistas_median']) == False) & (np.isnan(row['postu_total']) == False)):
        return row['postu_total']*row['vistas_median']
    elif((np.isnan(row['total_vistas'])) & (np.isnan((row['vistas_median']))) & (np.isnan((row['postu_total']))==False)):
        return row['postu_total']*a
    elif((np.isnan(row['total_vistas']) == False) & (np.isnan(row['postu_total']) == False) & (np.isnan(row['vistas_median']) == False)):
        if(row['total_vistas']<(row['postu_total']*row['vistas_median'])):
            return row['postu_total']*row['vistas_median']
    elif((np.isnan(row['total_vistas']) == False)&(np.isnan(row['postu_total']) == False)&(np.isnan(row['vistas_median']))):
        if(row['total_vistas']<row['postu_total']):
            return row['postu_total']
        
    return row['total_vistas']

In [79]:
df_avisos_detalles_final['total_vistas'] = df_avisos_detalles_final.apply(fill_total_vistas, axis=1)

In [79]:
df_avisos_detalles_final.isnull().sum()

idaviso                      0
tipo_de_trabajo              0
nivel_laboral                0
nombre_area                  0
denominacion_empresa         0
web                          0
crecimiento                  0
capacitacion                 0
clima laboral                0
multinacional                0
incorp_inmediata             0
disponibilidad_horaria       0
disponibilidad_viajar        0
flexibilidad                 0
secundario                   0
terciario                    0
ingles                       0
estudiantes                  0
conoc_informatica            0
experiencia                  0
proactivo                    0
relaciones_pers              0
desarrollador                0
cajero                       0
vendedor                     0
ingeniero                    0
analista                     0
administrativo               0
junior                       0
jefe                         0
marketing                    0
supervisor                   0
online  

In [81]:
df_avisos_detalles_final.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,...,administrativo,junior,jefe,marketing,supervisor,online,postu_total,total_vistas,vistas_median,cant_dias
0,8725750,1,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0.0,137.0,137.0,NaN,26.0
1,17903700,1,1,2,2,0,0,0,0,0,...,0,0,0,0,0,1.0,175.0,1935.0,2.0,51.0
2,1000150677,1,1,3,3,0,0,0,0,0,...,0,0,0,0,0,0.0,NaN,2.0,2.0,3.0
3,1000610287,1,1,3,4,0,1,1,0,1,...,0,0,0,0,0,1.0,197.0,394.0,2.0,83.0
4,1000872556,1,1,4,5,0,0,0,0,0,...,0,0,0,0,1,0.0,NaN,1.0,1.0,0.0


In [80]:
df_avisos_detalles_final.to_csv('Predecir/avisos_detalles_train.csv')

# Postulantes

In [81]:
df_postulantes_edu = pd.concat([df_postulantes_edu_original, df_postulantes_edu_nuevo])

In [82]:
df_postulantes_edu.shape

(705684, 3)

In [83]:
df_postulantes_edu.columns

Index(['idpostulante', 'nombre', 'estado'], dtype='object')

In [84]:
df_postulantes_edu.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [85]:
df_postulantes_edu['nombre'].value_counts()

Universitario        259550
Secundario           246012
Terciario/Técnico    114815
Otro                  53513
Posgrado              20885
Master                10297
Doctorado               612
Name: nombre, dtype: int64

In [86]:
df_postulantes_edu = df_postulantes_edu.drop_duplicates()

In [87]:
df_postulantes_edu.shape

(568612, 3)

In [88]:
nivel_dict = {'Universitario' : 3
             , 'Secundario' : 1
             , 'Terciario/Técnico' : 2
             , 'Otro': 0
             , 'Posgrado' : 4
             , 'Master' : 5
             , 'Doctorado' : 6}

def educacion(row):
    return nivel_dict[row['nombre']]

df_postulantes_edu['nombre'] = df_postulantes_edu.apply(educacion, axis=1)

In [89]:
df_postulantes_edu['estado'].value_counts()

Graduado      370213
En Curso      149388
Abandonado     49011
Name: estado, dtype: int64

In [90]:
estado_dict = {'Graduado' : 2
             , 'En Curso' : 1
             , 'Abandonado' : 0}

def estado(row):
    return estado_dict[row['estado']]

df_postulantes_edu['estado'] = df_postulantes_edu.apply(estado, axis=1)

In [91]:
df_postulantes_edu.shape

(568612, 3)

In [92]:
df_postulantes_edu['idpostulante'].nunique()

366395

Nos quedamos unicamente con su mayor nivel de estudio

In [93]:
for_join = df_postulantes_edu.loc[:, ['idpostulante', 'nombre']].groupby('idpostulante').max().reset_index()
df_postulantes_edu = pd.merge(df_postulantes_edu, for_join, how='inner', on=['idpostulante', 'nombre'])

In [94]:
df_postulantes_edu.shape

(387698, 3)

In [95]:
df_postulantes_edu['idpostulante'].nunique()

366395

In [96]:
df_postulantes_edu.head()

,idpostulante,nombre,estado
0,NdJl,4,1
1,1d2B,3,1
2,NPBx,5,1
3,ZKl5,5,1
4,Y043Z,3,1


In [97]:
for_join = df_postulantes_edu.loc[:, ['idpostulante', 'estado']].groupby('idpostulante').max().reset_index()
df_postulantes_edu = pd.merge(df_postulantes_edu, for_join, how='inner', on=['idpostulante', 'estado'])

In [98]:
df_postulantes_edu['idpostulante'].nunique()

366395

In [99]:
df_postulantes_datos = pd.concat([df_postulantes_datos_original, df_postulantes_datos_nuevo])
df_postulantes_datos = df_postulantes_datos.drop_duplicates()

In [100]:
df_postulantes_datos.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [101]:
df_postulantes_datos['idpostulante'].nunique()

408146

In [102]:
df_postulantes_datos['edad'] = pd.to_datetime(df_postulantes_datos["fechanacimiento"], errors = 'coerce')
del df_postulantes_datos['fechanacimiento']

In [103]:
df_postulantes_datos['idpostulante'].nunique()

408146

In [104]:
list_sexos = []

def tipo_sexos(row):
    if pd.isnull(row['sexo']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['sexo'] in list_sexos:
        return list_sexos.index(row['sexo'])+1
    
    list_sexos.append(row['sexo'])
    
    return list_sexos.index(row['sexo'])+1

df_postulantes_datos['sexo'] = df_postulantes_datos.apply(tipo_sexos, axis=1)

In [105]:
def edad(row):
    return 2018-row['edad'].year

df_postulantes_datos['edad'] = df_postulantes_datos.apply(edad, axis=1)

In [106]:
df_postulantes_datos.shape

(408452, 3)

In [107]:
df_postulantes_datos = df_postulantes_datos.drop_duplicates('idpostulante')

In [108]:
df_postulantes_datos.shape

(408146, 3)

In [109]:
df_postulantes_datos['idpostulante'].nunique()

408146

In [110]:
df_postulantes_edu['idpostulante'].nunique()

366395

In [111]:
df_postulantes = pd.merge(df_postulantes_edu, df_postulantes_datos, how='outer', on=['idpostulante'])

In [112]:
df_postulantes['idpostulante'].nunique()

408146

In [113]:
df_postulantes = df_postulantes.drop_duplicates('idpostulante')

In [114]:
df_postulantes.head()

,idpostulante,nombre,estado,sexo,edad
0,NdJl,4.0,1.0,2,49.0
1,1d2B,3.0,1.0,2,42.0
2,NPBx,5.0,1.0,2,37.0
3,ZKl5,5.0,1.0,2,44.0
4,Y043Z,3.0,1.0,2,39.0


In [115]:
df_postulantes.shape

(408146, 5)

In [116]:
vistas.head()

,idaviso,fechapostulacion,idpostulante
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP


In [117]:
vistas2 = vistas.loc[:, ['idaviso', 'idpostulante']].groupby('idpostulante').count().reset_index()
vistas2 = vistas2.rename(columns={'idaviso' : 'vistas_postulante'})

In [118]:
df_postulantes = pd.merge(df_postulantes, vistas2, on=['idpostulante'], how='left')

In [119]:
df_postulantes.head()

,idpostulante,nombre,estado,sexo,edad,vistas_postulante
0,NdJl,4.0,1.0,2,49.0,60.0
1,1d2B,3.0,1.0,2,42.0,NaN
2,NPBx,5.0,1.0,2,37.0,16.0
3,ZKl5,5.0,1.0,2,44.0,41.0
4,Y043Z,3.0,1.0,2,39.0,12.0


Considerar agregar vistas por dia!

In [120]:
fechas_postulaciones = df_postulaciones.loc[:, ['idpostulante', 'fechapostulacion']].groupby('idpostulante').min().reset_index()

In [121]:
vistas = vistas.rename(columns={'timestamp' : 'fechapostulacion'})

In [122]:
fechas_vistas_postul = vistas.loc[:, ['idpostulante', 'fechapostulacion']].groupby('idpostulante').min().reset_index()

In [123]:
fechas_iniciales = pd.concat([fechas_postulaciones, fechas_vistas_postul])

In [124]:
fechas_iniciales.shape

(655087, 2)

In [125]:
fechas_iniciales = fechas_iniciales.groupby('idpostulante').min().reset_index()

In [126]:
fechas_iniciales.shape

(393425, 2)

In [127]:
fechas_iniciales = fechas_iniciales.rename(columns={'fechapostulacion' : 'fecha_inicio_actividad'})

In [128]:
fechas_postulaciones = df_postulaciones.loc[:, ['idpostulante', 'fechapostulacion']].groupby('idpostulante').max().reset_index()

In [129]:
fechas_vistas_postul = vistas.loc[:, ['idpostulante', 'fechapostulacion']].groupby('idpostulante').max().reset_index()

In [130]:
fechas_finales = pd.concat([fechas_postulaciones, fechas_vistas_postul])

In [131]:
fechas_finales.shape

(655087, 2)

In [132]:
fechas_finales = fechas_finales.groupby('idpostulante').max().reset_index()

In [133]:
fechas_finales.shape

(393425, 2)

In [134]:
fechas_finales = fechas_finales.rename(columns={'fechapostulacion' : 'fecha_fin_actividad'})

In [135]:
fechas_totales = pd.merge(fechas_finales, fechas_iniciales, on=['idpostulante'], how='inner')

In [136]:
fechas_totales.head()

,idpostulante,fecha_fin_actividad,fecha_inicio_actividad
0,0z5Dmrd,2018-03-28 20:25:03.849,2018-03-28 16:24:37
1,0z5JW1r,2018-04-16 13:39:49.985,2018-01-16 07:45:20
2,0z5VvGv,2018-04-01 02:02:18.809,2018-01-18 10:00:58
3,0zB01pE,2018-04-10 01:03:21.756,2018-04-09 21:00:48
4,0zB026d,2018-03-28 20:53:38.480,2018-02-01 17:00:24


In [137]:
fechas_totales['cant_dias_actividad'] = (fechas_totales['fecha_fin_actividad']-fechas_totales['fecha_inicio_actividad']).apply(set_dias)

In [138]:
fechas_totales.head()

,idpostulante,fecha_fin_actividad,fecha_inicio_actividad,cant_dias_actividad
0,0z5Dmrd,2018-03-28 20:25:03.849,2018-03-28 16:24:37,0
1,0z5JW1r,2018-04-16 13:39:49.985,2018-01-16 07:45:20,90
2,0z5VvGv,2018-04-01 02:02:18.809,2018-01-18 10:00:58,72
3,0zB01pE,2018-04-10 01:03:21.756,2018-04-09 21:00:48,0
4,0zB026d,2018-03-28 20:53:38.480,2018-02-01 17:00:24,55


In [139]:
del fechas_totales['fecha_fin_actividad']
del fechas_totales['fecha_inicio_actividad']

In [140]:
df_postulantes = pd.merge(df_postulantes, fechas_totales, on=['idpostulante'], how='left')

In [139]:
df_postulantes.shape

(408146, 7)

Agregamos tipo de aviso más popular por postulante, empresa popular y nombre de area popular

In [140]:
df_avisos_detalles_final.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,...,administrativo,junior,jefe,marketing,supervisor,online,postu_total,total_vistas,vistas_median,cant_dias
0,8725750,1,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0.0,137.0,137.0,NaN,26.0
1,17903700,1,1,2,2,0,0,0,0,0,...,0,0,0,0,0,1.0,175.0,1935.0,2.0,51.0
2,1000150677,1,1,3,3,0,0,0,0,0,...,0,0,0,0,0,0.0,NaN,2.0,2.0,3.0
3,1000610287,1,1,3,4,0,1,1,0,1,...,0,0,0,0,0,1.0,197.0,394.0,2.0,83.0
4,1000872556,1,1,4,5,0,0,0,0,0,...,0,0,0,0,1,0.0,NaN,1.0,1.0,0.0


In [141]:
df_postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


In [144]:
postulaciones_total = postulaciones.loc[:, ['idaviso', 'idpostulante']].rename(columns={'idaviso':'postu_postulante'})\
                .groupby('idpostulante').count().reset_index()

In [142]:
df_postulantes.head()

,idpostulante,nombre,estado,sexo,edad,vistas_postulante,cant_dias_actividad
0,NdJl,4.0,1.0,2,49.0,60.0,88.0
1,1d2B,3.0,1.0,2,42.0,NaN,7.0
2,NPBx,5.0,1.0,2,37.0,16.0,81.0
3,ZKl5,5.0,1.0,2,44.0,41.0,89.0
4,Y043Z,3.0,1.0,2,39.0,12.0,50.0


In [145]:
df_postulantes = pd.merge(df_postulantes, postulaciones_total, on=['idpostulante'], how='left')

In [149]:
def fill_vistas_postu(row):
    if((np.isnan(row['vistas_postulante'])) & (np.isnan(row['postu_postulante']) == False)):
        return row['postu_postulante']
    elif((np.isnan(row['vistas_postulante']) == False) & (np.isnan(row['postu_postulante']) == False)):
        if(row['postu_postulante']>row['vistas_postulante']):
            return row['postu_postulante']
        
    return row['vistas_postulante']

In [150]:
df_postulantes['vistas_postulante'] = df_postulantes.apply(fill_vistas_postu, axis=1)

In [151]:
df_postulantes.head()

,idpostulante,nombre,estado,sexo,edad,vistas_postulante,cant_dias_actividad,postu_postulante
0,NdJl,4.0,1.0,2,49.0,60.0,88.0,30.0
1,1d2B,3.0,1.0,2,42.0,4.0,7.0,4.0
2,NPBx,5.0,1.0,2,37.0,34.0,81.0,34.0
3,ZKl5,5.0,1.0,2,44.0,41.0,89.0,11.0
4,Y043Z,3.0,1.0,2,39.0,12.0,50.0,4.0


In [152]:
df_postulantes.to_csv('Predecir/postulantes_datos_train.csv')

# Postulaciones con postulantes (To train)

In [70]:
df_postulaciones = pd.concat([df_postulaciones_nuevo, df_postulaciones_original])

In [71]:
df_postulaciones.shape

(8311264, 3)

In [72]:
df_postulaciones.columns

Index(['idaviso', 'idpostulante', 'fechapostulacion'], dtype='object')

Elimino duplicados

In [73]:
df_postulaciones = df_postulaciones.drop_duplicates()

In [74]:
df_postulaciones.shape

(6604534, 3)

In [75]:
df_postulaciones['fechapostulacion'] = pd.to_datetime(df_postulaciones["fechapostulacion"], errors = 'coerce')

In [79]:
df_postulaciones['fechapostulacion'].min()

Timestamp('2018-01-15 00:00:01')

In [78]:
df_postulaciones['fechapostulacion'].max()

Timestamp('2018-04-17 10:42:39')

In [82]:
def get_hour(row):
    return row['fechapostulacion'].hour

def get_day(row):
    return row['fechapostulacion'].day

def get_month(row):
    return row['fechapostulacion'].month


df_postulaciones['hora'] = df_postulaciones.apply(get_hour, axis=1)
df_postulaciones['dia'] = df_postulaciones.apply(get_day, axis=1)
df_postulaciones['mes'] = df_postulaciones.apply(get_month, axis=1)

In [84]:
del df_postulaciones['fechapostulacion']

In [85]:
df_postulaciones.head()

,idaviso,idpostulante,hora,dia,mes
0,1112248724,NjlD,7,19,1
1,1112286523,ZaO5,15,24,1
2,1112272060,ZaO5,15,24,1
3,1112288401,ZaO5,8,26,1
4,1112300563,ZaO5,13,30,1


In [86]:
df_postulaciones.to_csv('df_postulaciones.csv')

In [87]:
df_postulaciones.shape

(6604534, 5)

In [88]:
del df_postulaciones['hora']
del df_postulaciones['dia']
del df_postulaciones['mes']

In [89]:
df_postulaciones.to_csv('Predecir/df_postulaciones.csv')